In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from SubjectiveLogic import Hyperopinion, HyperopinionInterface
from SubjectiveLogic import Hyperopinion as W
from SubjectiveLogic.BeliefFusion import cumulative_fusion, averaging_fusion, weighted_fusion
from SubjectiveLogic.Trust import trust_discount_2e
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Callable

# Opinion representation

A *belief state* in the polarization paper is $\mathrm{Bel}_p : A \rightarrow [0, 1]$. Would be trivial if we had only 1 degree of freedom for subjective logic's binomial opinions, but we have 2.

**Definition:** *(Multinomial opinion)* Let $\mathbb{X}$ be a domain and $k = |X|$. Let $X$ be a random variable in $\mathbb{X}$. A multinomial opinion over the random variable $X$ is the ordered triplet $\omega_X = (\mathbf{b}_X, u_X , \mathbf{a}_X)$ where

- $\mathbf{b}_X$ is a belief mass distribution over $X$,
- $u_X$ is the uncertainty mass which represents the vacuity of evidence,
- $\mathbf{a}_X$ is a base rate distribution over $\mathbb{X}$,

and $\sum_{x \in \mathbb{X}} \mathbf{b}_X(x) + u_X = 1$.

<!-- **Definition:** *(Binomial opinion)* Let $\mathbb{X} = \{x, \overline{x}\}$ be a binary domain with binomial random variable $X \in \mathbb{X}$. A binomial opinion over the random variable $X$ is the ordered triplet $\omega_X = (\mathbf{b}_X, u_X , \mathbf{a}_X)$ where
- $\mathbf{b}_X$ is a belief mass distribution over $X$,
- $u_X$ is the uncertainty mass which represents the vacuity of evidence,
- $\mathbf{a}_X$ is a base rate distribution over $\mathbb{X}$, -->

The projected probability distribution of multinomial opinions is defined by:
$$\mathbf{P}_X(x) = \mathbf{b}_X(x) + \mathbf{a}_X(x) u_X,\ \forall x \in \mathbb{X}\text{.}$$

In [3]:
# Hyperopinion(2, [b1, b2])

# Aleatory and epistemic opinions (from section 3.3 of Jøsang's book)

An important aspect of uncertainty quantification in the scientific literature is the distinction between aleatory and epistemic uncertainty, described below.

- _Aleatory uncertainty_ which is the same as statistical uncertainty, expresses that we do not know the outcome each time we run the same experiment, we only know the long-term relative frequency of outcomes. For example, flipping a coin, which could be unfair.
- _Epistemic uncertainty_ expresses that we could in principle know the outcome of a specific or future or past event, but that we do not have enough evidence to know it exactly. For example, the assassination of President Kennedy in 1963 is believed by most people to have been committed by Lee Harvey Oswald, but there is considerable uncertainty around it.

Note that both aleatory and epistemic uncertainty represent _first-order_ uncertainty, and therefore are not the same type of uncertainty as the uncertainty mass in opinions, which represents _second-order uncertainty_. Uncertainty of a binary variable are greatest with $p = \frac{1}{2}$. In contrast, when an opinion has projected probability $\mathbf{P}_X(x) = \frac{1}{2}$ for some value $x$, it says nothing about the amount of uncertainty mass $u_X$ in the opinion.

This distinction between aleatory and epistemic (first-order) uncertainty puts constraints on the opinion classes that naturally apply to specific situation.

- _An aleatory opinion_ applies to a variable governed by a frequentist process, and that represents the (uncertain) likelihood of values of the variable in any unknown past or future instance of the process. An aleatory opinion can naturally have an _arbitrary uncertainty mass_.
- _An epistemic opinion_ applies to a variable that is assumed to be non-frequentist, and that represents the (uncertain) likelihood of values of the variable in a specific unknown past or future instance. An aleatory opinion is naturally _constrained to be uncertainty-maximised_.

The reason why epistemic opinions should be uncertainty-maximised is that epistemic evidence can not be accumulated in a statistical manner, which would reduce the uncertainty mass. Instead, different pieces of epistemic evidence that support opposite/different values should cancel each other out. (❗ Like Mário said. I need to find out the properties of belief fusion operators when fusing epistemic opinions.)

For social networks, the polarization paper uses epistemic propositions, thus I may be focusing on epistemic opinions.

To reproduce the beliefs from the polarization paper, we need to use binomial opinions. So when we say *opinion*, we are talking about a *binomial opinion*, a *multinomial opinion* with $k = 2$.

TODO: Justify the use of epistemic opinions vs aleatory opinions.

# Uncertainty maximization (optional)

Uncertainty maximization is the process to maximize uncertainty mass while keeping the same projected probability distribution. The corresponding uncertainty-maximized opinion is denoted $\ddot{\omega}_X = (\ddot{\mathbf{b}}_X, \ddot{u}_X, \ddot{\mathbf{a}}_X)$.

**Definition:** _(Uncertainty maximization)_ Let $\omega_X = (\mathbf{b}_X, u_X, \mathbf{a}_X)$ be an opinion. The uncertainty maximized opinion from $\omega_X$ is 
$$\ddot{\omega}_X = (\ddot{\mathbf{b}}_X, \ddot{u}_X, \ddot{\mathbf{a}}_X)$$
such that
- $\ddot{u}_X = \min\limits_{x \in \mathbb{X}} \left(\dfrac{\mathbf{P}_X(x)}{\mathbf{a}_X(x)}\right)$
- $\ddot{\mathbf{b}}_X(x) = \mathbf{P}_X(x) - \mathbf{a}_X(x) \ddot{u}_X$, for $x \in \mathbb{X}$
- $\ddot{\mathbf{a}}_X = \mathbf{a}_X$

In the case of hyper-opinions, there is no simple method for redistributing belief mass from singletons (❗ unit subset of the domain) and composite values (❗ proper subset of the domain) and composite values to form increased uncertainty mass in a consistent way. It is not meaningful to distinguish between aleatory and epistemic opinions in the case of hyper-opinions. (❓ I'm not sure why)

In [4]:
A = Hyperopinion(2, [0.7, 0.3])
A.P # Projected probability of A

array([0.7, 0.3])

In [5]:
A_ = A.maximize_uncertainty()
A_ # A uncertainty maximized

Hyperopinion(2, [0.4 0. ], [0.5 0.5])

In [6]:
A.P # Same projected probability after uncertainty maximization

array([0.7, 0.3])

# Categories of opinions

Here I describe categories of opinions we will use.

## Epistemic opinion

If $\mathrm{Bel}_X(A) = B$, then
$$
\omega^A_X = 
\left\{\begin{array}{lrlll}
((0, & - 2B + 1), & 2B, & \mathbf{a}_X), & \text{if } B < 0.5 \\
((2B - 1, & 0), & 2 - 2B, & \mathbf{a}_X), & \text{if } B \geq 0.5
\end{array}\right.
$$

**Example:** $\mathrm{Bel}^A_X = 0.25$.

$\omega^A_X = ((0, 0.5), 0.5, \mathbf{a}_X)$.

In [22]:
print(Hyperopinion(2, [0, 0.5]))

k = 2
            
b({0}) = 0.0
b({1}) = 0.5
u = 0.5

a({0}) = 0.5
a({1}) = 0.5

P({0}) = 0.25
P({1}) = 0.75


**Example:** $\mathrm{Bel}^A_X = 0.75$.

$\omega^A_X = ((0.5, 0), 0.5, \mathbf{a}_X)$.

In [8]:
Hyperopinion(2, [0.5, 0])

Hyperopinion(2, [0.5 0. ], [0.5 0.5])

## Dogmatic opinion

A *dogmatic opinion* is an opinion such that $u_x = 0$. When it's not absolute, i.e. $\mathbf{b}_X(x) \neq 1$. then it must be a aleatory opinion. A dogmatic opinion corresponds to the probability of $x$ in probabilistic logic.

If $\mathrm{Bel}_X(A) = B$, then $\omega^A_X = ((B, 1 - B), 0, \mathbf{a}_X)$.

**Example:** $\mathrm{Bel}^A_X = 0.3$.

$\omega^A_X = ((0.3, 0.7), 0, \mathbf{a}_X)$. Agent $A$ agrees by $30\%$ and disagrees by $70\%$.

In [9]:
Hyperopinion(2, [0.3, 0.7])

Hyperopinion(2, [0.3 0.7], [0.5 0.5])

**Example:** "_A coin is biased to show heads by 70%_" is an aleatory opinion and dogmatic because there is no uncertainty about the coin's bias.

$\omega_X = ((0.7, 0.3), 0, \mathbf{a}_X)$

In [10]:
Hyperopinion(2, [0.7, 0.3])

Hyperopinion(2, [0.7 0.3], [0.5 0.5])

## Zero disbelief opinion

If $\mathrm{Bel}_X(A) = B$, then $\omega^A_X = ((B, 0), 1 - B, \mathbf{a}_X)$.

**Example:** $\mathrm{Bel}_X(A) = 0.3$.

$\omega^A_X = ((0.3, 0), 0.7, \mathbf{a}_X)$. Agent $A$ agrees by $30\%$ and is uncertain by $70\%$.

In [11]:
Hyperopinion(2, [0.3, 0])

Hyperopinion(2, [0.3 0. ], [0.5 0.5])

## Non-dogmatic fixed uncertainty

If $\mathrm{Bel}_X(A) = B$, then $\omega^A_X = ((B(1 - u_X), (1 - B)(1 - u_X)), u_X, \mathbf{a}_X)$, for some $0 < u_X < 1$. This is useful for simulations with belief fusion when comparing with dogmatic opinion.

**Example:** $\mathrm{Bel}_X(A) = 0.5$. $u_X = 0.1$

$\omega^A_X = ((0.45, 0.45), 0.1, \mathbf{a}_X)$. Agent $A$ agrees by $45\%$, disagrees by $45\%$, and is uncertain by $10\%$.

In [12]:
Hyperopinion(2, [0.45, 0.45])

Hyperopinion(2, [0.45 0.45], [0.5 0.5])

# Update function

## Mapping between opinion and Dirithlet PDF

**To do:** Explain what it means *amount of evidence* when talking about opinions. This is necessary for understanding belief fusion.



## Belief fusion

**To do:** Define the belief fusion operator, their properties, and what does that mean in terms of amount of evidence.

In [13]:
AX = Hyperopinion(2, [0.3, 0])
BX = Hyperopinion(2, [0.7, 0])

### Cumulative fusion

**To do:** Sum of evidences.

In [19]:
cumulative_fusion

<function SubjectiveLogic.BeliefFusion.cumulative_fusion(CC)>

**Property:** _It it not idempotent: Equal non-dogmatic opinions will increase confidence. E.g.:

In [21]:
AX = Hyperopinion(2, [0.5, 0])
BX = Hyperopinion(2, [0.5, 0])
cumulative_fusion([AX, BX])

Hyperopinion(2, [0.66666667 0.        ], [0.5 0.5])

**Properties:**
- It is not idempotent: Equal non-dogmatic opinions will increase confidence.
- If there is a dogmatic opinion, the opinion will be an "average" between the dogmatic opinions.

### Averaging fusion

**To do:** Average of evidences.

In [15]:
averaging_fusion([AX, BX])

Hyperopinion(2, [0.51 0.49], [0.5 0.5])

**Properties:**
- It's idempotent: Equal non-dogmatic opinions have the same confidence.
- But it doesn't have a neutral element: Fusing a completely uncertain opinion with a non-dogmatic opinion will decrease confidence.
- If there is a dogmatic opinion, the opinion will be an "average" between the dogmatic opinions.

### Weighted fusion

**To do:** Average of evidences weighted by confidence.

In [16]:
weighted_fusion([AX, BX])

Hyperopinion(2, [0.51 0.49], [0.5 0.5])

**Properties:**
- It's idempotent: Equal non-dogmatic opinions have the same confidence.
- It has a neutral element: Fusing a completely uncertain opinion with a non-dogmatic opinion will have the same belief mass as the non-dogmatic opinion.
- If there is a dogmatic opinion, the opinion will be an "average" between the dogmatic opinions.

## Trust discount

**To do:** Define trust discount operator.

In [18]:
trust_discount_2e(AB

<function SubjectiveLogic.Trust.trust_discount_2e(hab: SubjectiveLogic.HyperopinionInterface.HyperopinionInterface, hbx: SubjectiveLogic.HyperopinionInterface.HyperopinionInterface)>

## Update function

Fuse A's opinion with what A learns by trusting others.

# Initial states

Show some initial states.

## Only dogmatic opinions

To show how dogmatic opinions doesn't change if trust is less than 1. Trust equal to 1 goes to average.

## Only fixed disbelief opinions

To show how opinions converge to vacuous (u = 1) if trust is less than 1. I need to test when trust is 1.

## Only non-dogmatic fixed uncertainty opinions

I didn't tried yet.

## Only one dogmatic opinion

Useful to show how dogmatic opinions remains unchanged and influence non-dogmatic opinions.

# Simulations

Lacração, marxismo e bruxaria.

## 

In [17]:
Hyperopinion(3, [1, 0, 0]).maximize_uncertainty()

Hyperopinion(3, [1. 0. 0. 0. 0. 0.], [0.33333333 0.33333333 0.33333333])

In [6]:
A = Hyperopinion(2, [0.3, 0], [0.5, 0.5])
print(A)

k = 2
            
b({0}) = 0.3
b({1}) = 0.0
u = 0.7

a({0}) = 0.5
a({1}) = 0.5

P({0}) = 0.6499999999999999
P({1}) = 0.35


In [7]:
print(Hyperopinion(2, [0.75, 0.25], [0.5, 0.5]).maximize_uncertainty())

k = 2
            
b({0}) = 0.5
b({1}) = 0.0
u = 0.5

a({0}) = 0.5
a({1}) = 0.5

P({0}) = 0.75
P({1}) = 0.25


In [9]:
def pa(pp, a):
    return Hyperopinion(2, [pp, 1 - pp], [a, 1 - a]).maximize_uncertainty()

In [16]:
for i in np.arange(0, 1, 0.1):
    print("b0", pa(i, 0.5).b[0])
    print("b1", pa(i, 0.5).b[1])
    print("p0", pa(i, 0.5).b[1])
    print()

b0 0.0
b1 1.0
p0 1.0

b0 0.0
b1 0.8
p0 0.8

b0 0.0
b1 0.6000000000000001
p0 0.6000000000000001

b0 0.0
b1 0.3999999999999999
p0 0.3999999999999999

b0 0.0
b1 0.19999999999999996
p0 0.19999999999999996

b0 0.0
b1 0.0
p0 0.0

b0 0.20000000000000018
b1 0.0
p0 0.0

b0 0.40000000000000013
b1 0.0
p0 0.0

b0 0.6000000000000001
b1 0.0
p0 0.0

b0 0.8
b1 0.0
p0 0.0



a 0.5 -> p 0.8
a 0.8 -> p 0.8 never talked

a b1 = p -> 

2 people 
base rate as original opinion
